In [9]:
import sys
sys.path.insert(1, 'C:\\Users\\oper\\SynologyDrive\\Lab2023\\Qubit\\QTLab2324\\PAandQubit\\Instruments\\network\\Signal_Generator')
sys.path.insert(2, 'C:\\Users\\oper\\SynologyDrive\\Lab2023\\Qubit\\QTLab2324\\PAandQubit\\Instruments\\Acquisition_Card')

import RS_SMA100B
from Card_Class import *


ip = '192.168.40.15' # Set IP address of the device
pulse_freq = 100e6 # Set frequency of the signal to send to the resonator
amplitude = -25 # Set the amplitude of the signal in dBm
pulse_time = 2 # 2 microsec
sGen = RS_SMA100B.SMA100B(ip)

pxie = PXIe5170R('PXI1Slot4')

sGen.reset()
sGen.clear()
sGen.RF_freq(pulse_freq) # in Hz
sGen.RF_lvl_ampl(amplitude)
sGen.pul_gen_params(period = pulse_time*2, delay = 3e6, width = pulse_time)
sGen.pul_gen_mode('SING')
sGen.pul_trig_mode('SING')


# configure parameters
pxie.voltage_range = 2.0
pxie.sample_rate = 2 * pulse_freq # f_sample > 2*f_max
pxie.coupling = 'AC' # or DC????
pxie.num_pts = 1000
pxie.num_records = 1
pxie.ref_position = 0

sGen.pul_state(1)
sGen.RF_state(1)


# configure the acquisition
pxie.acquisition()
sGen.pul_exe_sing_trig() # DA TESTARE

Connection successful!


DriverError: -1074126845: Maximum time exceeded before the operation completed.

Device: PXI1Slot4


In [14]:




# pxie.available

# configure parameters
pxie.voltage_range = 3
pxie.voltage_range
# pxie.sample_rate = 2 * pulse_freq # f_sample > 2*f_max
# pxie.coupling = 'AC' # or DC????
# pxie.num_pts = # ??
# pxie.num_records = 1
# pxie.ref_position = 50.0

# configure the acquisition
print(pxie.acquisition())

3